# Sentiment Analysis with Wasm and SingleStore

## Connect to SingleStore

In [56]:
import plotly.express as px
from pymysql import connect, cursors
from dotenv import load_dotenv
import pandas as pd
from pathlib import Path
load_dotenv(dotenv_path=Path('.') / '.secrets.env')

SINGLESTORE_HOST = os.environ.get('SINGLESTORE_HOST')
SINGLESTORE_USER = os.environ.get('SINGLESTORE_USER')
SINGLESTORE_PASSWORD = os.environ.get('SINGLESTORE_PASSWORD')
SINGLESTORE_PORT = 3306
SINGLESTORE_URI = "mysql+mysqldb://{}:{}@{}:{}/demo".format(SINGLESTORE_USER, SINGLESTORE_PASSWORD, SINGLESTORE_HOST, SINGLESTORE_PORT)

s2 = connect(host=SINGLESTORE_HOST, port=SINGLESTORE_PORT, user=SINGLESTORE_USER, passwd=SINGLESTORE_PASSWORD, db='demo', cursorclass=cursors.DictCursor)
cursor = s2.cursor()

## Inspect the data

StackOverflow data. Did you know comments also have a score like questions and answers? Let's analyze 82 million rows of comments.

What do you think the sentiment of highly up-voted comments is?

In [57]:
cursor.execute("select count(*) as cnt from comments")
cursor.fetchone()["cnt"]

82037742

In [58]:
df = pd.read_sql("select * from comments limit 10;", SINGLESTORE_URI)
df.head()

,id,text,creation_date,score,score_bucket
0,54613446,"Hi Mark, I have tried that and it didn't work ...",2015-10-29 10:27:04,0,0
1,54615167,"No, I am starting a bounty on this one. Still ...",2015-10-29 11:13:47,0,0
2,54613090,@maryisdead He wants to red part of image be a...,2015-10-29 10:18:40,0,0
3,54612491,Why `count($testas)` (just variable) but `$obj...,2015-10-29 10:04:53,0,0
4,54612237,I end up with images that look like this http:...,2015-10-29 09:59:09,0,0


### Create a sentiment analysis function

A few pre-reqs were performed prior to this analysis.

1. We generated bindings for [vader-sentiment-rust](https://github.com/ckw017/vader-sentiment-rust) and compiled the original crate and bindings with `cargo wasi build --release`.
2. Uploaded the wasm module to cloud storage, for this example, we used GCS.
3. Then via SQL we create the function using a web link.

In [ ]:
cursor.execute("""
    create function sentimentable returns table as wasm
        from link gcs_wasm 'wasm-modules/sentimentable.wasm'
        with wit from link gcs_wasm 'wasm-modules/sentimentable.wit';
""")

In [60]:
cursor.execute("select * from sentimentable('read the docs')")
cursor.fetchall()

[{'compound': 0.0, 'positive': 0.0, 'negative': 0.0, 'neutral': 1.0}]

In [61]:
df = pd.read_sql("""select
    score_bucket,
    count(*) as num_comments,
    abs(min(sentiment.compound)) as "negative",
    max(sentiment.compound) as "positive"
from (
    select score_bucket, text
    from comments
    where score >= 10
) as c
join sentimentable(c.text) as sentiment
group by 1
having positive > 0 and negative > 0 and num_comments > 20
order by 1 asc;""", SINGLESTORE_URI)
df.head()

,score_bucket,num_comments,negative,positive
0,10,115285,0.999408,0.988480
1,20,27409,0.988882,0.989240
2,30,12064,0.982151,0.987701
3,40,6701,0.984391,0.987944
4,50,4130,0.957839,0.977145


In [62]:
colorscheme = ["#00E676" ,"#AA00FF"]

fig = px.scatter(df,
    x="score_bucket",
    y=["positive", "negative"],
    trendline="ols",                        # ols = Ordinary Least Squares
    color_discrete_sequence=colorscheme,
    template="plotly_dark",
    labels={
        "score_bucket": "Comment Score",
        "value": "polarization",
    }
)

fig.update_layout(
    paper_bgcolor='#1B1A21',
    plot_bgcolor='#1B1A21',
)

fig.show()